In [1]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [2]:
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }
prices = pd.read_csv("sell_prices.csv", dtype = PRICE_DTYPES)
prices.head(1)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58


In [3]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
cal = pd.read_csv("calendar.csv", dtype = CAL_DTYPES)
cal.head(1)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [4]:
dt = pd.read_csv("sales_train_validation.csv")
dt.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1


In [5]:
h = 28 
max_lags = 57
last_day = 1913  # 历史的最后一天
fday = datetime(2016,4, 25)  # 预测的第一天
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [6]:
# first_day为预测的第一天

def create_dt(first_day = 1200):
    prices = pd.read_csv("sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")  # 类似于labelencoder，将store_id和item_id数字化表示
            prices[col] -= prices[col].min()  
            
    cal = pd.read_csv("calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])  #  转化为datetime类型
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")  # 将event_name_1/event_type_1/event_name_2/event_type_2weekday数字化表示
            cal[col] -= cal[col].min()  # NaN会转换为-1，该代表将NaN归为0，其他从1开始
    
    # 处理train_validation文件
    numcols = [f"d_{day}" for day in range(first_day, last_day + 1)]  # 选取历史数据，从first_day到tr_last（包含这一天）d_1200,d_1201
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol: "float32" for numcol in numcols}   # d_1200为float32
    
    dtype.update({col: "category" for col in catcols if col != "id"})  # 除id外，都为category类型
    dt = pd.read_csv("sales_train_validation.csv", usecols = catcols + numcols, dtype = dtype)  # 取catcols和numcols的列(只取了需要的历史天数销量数据)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")  # 数字化
            dt[col] -= dt[col].min()
    
    
    for day in range(last_day + 1, last_day + 28 +1):  # 遍历预测的每一天
        dt[f"d_{day}"] = np.nan  # 先填充为NaN
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [7]:
FIRST_DAY = 350

In [8]:
df = create_dt(first_day = FIRST_DAY)

In [9]:
df.head()  # 将train_validation和calendar按d拼接，然后再与price按store_id/item_id/wm_yr_wk进行拼接

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1500,0.0,2015-03-08,11506,...,3,2015,0,0,0,0,1.0,0.0,1.0,8.26
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1501,0.0,2015-03-09,11506,...,3,2015,0,0,0,0,1.0,1.0,1.0,8.26
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1502,0.0,2015-03-10,11506,...,3,2015,0,0,0,0,1.0,0.0,0.0,8.26
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1503,1.0,2015-03-11,11506,...,3,2015,0,0,0,0,0.0,1.0,1.0,8.26
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1504,1.0,2015-03-12,11506,...,3,2015,0,0,0,0,0.0,1.0,1.0,8.26


In [15]:
df[["id","sales"]].groupby("id")["sales"].shift(7)

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
           ... 
12567119    0.0
12567120    0.0
12567121    1.0
12567122    0.0
12567123    0.0
Name: sales, Length: 12567124, dtype: float32

In [16]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):  # [(7, lag_7), (28, lag_28)]
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)  # 新增两列，同一商品前7天的单价和前28天的价格

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean()) 

            # log=7，wins=28表示一个商品当前的参考价为：从当前时间7天前（如1.8号的七天前是1.1号）开始一共向上取28天，求这些天的均值
    
    date_features = {
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",

    }
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [17]:
create_fea(df)

In [18]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1500,0.0,2015-03-08,11506,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,10,1,8
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1501,0.0,2015-03-09,11506,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,11,1,9
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1502,0.0,2015-03-10,11506,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,11,1,10
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1503,1.0,2015-03-11,11506,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,11,1,11
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1504,1.0,2015-03-12,11506,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,11,1,12


In [19]:
df.dropna(inplace = True)
df.shape

(10890174, 31)

In [20]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday", 'event_name_1', 'event_name_2']
train_cols = df.columns[~df.columns.isin(useless_cols)]
print(train_cols)
X_train = df[train_cols]
y_train = df["sales"]

Index(['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'wday', 'month',
       'year', 'event_type_1', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
       'sell_price', 'lag_7', 'lag_28', 'rmean_7_7', 'rmean_28_7',
       'rmean_7_28', 'rmean_28_28', 'week', 'quarter', 'mday'],
      dtype='object')


In [21]:
X_train.head()

,item_id,dept_id,store_id,cat_id,state_id,wday,month,year,event_type_1,event_type_2,...,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
1650761,0,0,0,0,0,1,5,2015,0,0,...,8.26,2.0,2.0,0.857143,0.857143,0.750000,0.785714,18,2,2
1650762,0,0,0,0,0,2,5,2015,0,0,...,8.26,0.0,0.0,0.857143,0.571429,0.678571,0.785714,18,2,3
1650763,0,0,0,0,0,3,5,2015,0,0,...,8.26,1.0,0.0,0.857143,0.428571,0.678571,0.785714,19,2,4
1650764,0,0,0,0,0,4,5,2015,1,0,...,8.26,0.0,1.0,0.857143,0.571429,0.678571,0.821429,19,2,5
1650765,0,0,0,0,0,5,5,2015,0,0,...,8.26,0.0,2.0,0.571429,0.857143,0.678571,0.857143,19,2,6


In [23]:
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                    weights='uniform')

In [24]:
alphas = [1.035, 1.03, 1.025]
weights = [1 / len(alphas)] * len(alphas)  # 权重，当前权重值一样
print(weights)

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]


In [25]:
# alphas：利用lgb预测出模型后，乘以alphas才为最终的预测值
# weight：由于有三个alphas，这三个alphas得出的结果各赋予一定的权重

sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):  # [(1.035, 0.333), (1.03, 0.333), (1.025, 0.333)]
    # te是所有的数据，tst是部分数据
    te = create_dt(False)  # is_train = False  原始数据
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)  # 预测的具体哪一天，比如第1914那天的日期
        print(icount, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()  
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]  # 需要预测的数据
        te.loc[te.date == day, "sales"] = alpha * model_rf.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()  # 取出预测的数据
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]  # [F1,F2,F3,...F28]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()  # 转变成submission的形式（只是index只有每种商品的预测值，没有分位数和汇总的一些）
    te_sub.fillna(0., inplace = True) 
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_rf_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission_rf.csv",index=False)

0 2016-04-25 00:00:00
0 2016-04-26 00:00:00
0 2016-04-27 00:00:00
0 2016-04-28 00:00:00
0 2016-04-29 00:00:00
0 2016-04-30 00:00:00
0 2016-05-01 00:00:00
0 2016-05-02 00:00:00
0 2016-05-03 00:00:00
0 2016-05-04 00:00:00
0 2016-05-05 00:00:00
0 2016-05-06 00:00:00
0 2016-05-07 00:00:00
0 2016-05-08 00:00:00
0 2016-05-09 00:00:00
0 2016-05-10 00:00:00
0 2016-05-11 00:00:00
0 2016-05-12 00:00:00
0 2016-05-13 00:00:00
0 2016-05-14 00:00:00
0 2016-05-15 00:00:00
0 2016-05-16 00:00:00
0 2016-05-17 00:00:00
0 2016-05-18 00:00:00
0 2016-05-19 00:00:00
0 2016-05-20 00:00:00
0 2016-05-21 00:00:00
0 2016-05-22 00:00:00
0 1.035 0.3333333333333333
1 2016-04-25 00:00:00
1 2016-04-26 00:00:00
1 2016-04-27 00:00:00
1 2016-04-28 00:00:00
1 2016-04-29 00:00:00
1 2016-04-30 00:00:00
1 2016-05-01 00:00:00
1 2016-05-02 00:00:00
1 2016-05-03 00:00:00
1 2016-05-04 00:00:00
1 2016-05-05 00:00:00
1 2016-05-06 00:00:00
1 2016-05-07 00:00:00
1 2016-05-08 00:00:00
1 2016-05-09 00:00:00
1 2016-05-10 00:00:00
1 201

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm as tqdm

from ipywidgets import widgets, interactive, interact
import ipywidgets as widgets
from IPython.display import display

import os

In [2]:
train_sales = pd.read_csv('sales_train_validation.csv')
calendar_df = pd.read_csv('calendar.csv')
submission_file = pd.read_csv('sample_submission.csv')
sell_prices = pd.read_csv('sell_prices.csv')
rf_sub = pd.read_csv("submission_rf.csv")

In [3]:
rf_col = ['id']
rf_col.extend([f'd_{day}' for day in range(1914, 1914+28)])
rf_sub.columns = rf_col

In [4]:
train_sales = train_sales.merge(rf_sub, on='id')

In [5]:
total = ['Total']
train_sales['Total'] = 'Total'
train_sales['state_cat'] = train_sales.state_id + "_" + train_sales.cat_id
train_sales['state_dept'] = train_sales.state_id + "_" + train_sales.dept_id
train_sales['store_cat'] = train_sales.store_id + "_" + train_sales.cat_id
train_sales['store_dept'] = train_sales.store_id + "_" + train_sales.dept_id
train_sales['state_item'] = train_sales.state_id + "_" + train_sales.item_id
train_sales['item_store'] = train_sales.item_id + "_" + train_sales.store_id

In [6]:
val_eval = ['validation', 'evaluation']

# creating lists for different aggregation levels
total = ['Total']
states = ['CA', 'TX', 'WI']
num_stores = [('CA',4), ('TX',3), ('WI',3)]
stores = [x[0] + "_" + str(y + 1) for x in num_stores for y in range(x[1])]
print(stores)  # 商店名

['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']


In [7]:
cats = ['FOODS', 'HOBBIES', 'HOUSEHOLD']
num_depts = [('FOODS',3), ('HOBBIES',2), ('HOUSEHOLD',2)]
depts = [x[0] + "_" + str(y + 1) for x in num_depts for y in range(x[1])]
state_cats = [state + "_" + cat for state in states for cat in cats]
state_depts = [state + "_" + dept for state in states for dept in depts]
store_cats = [store + "_" + cat for store in stores for cat in cats]
store_depts = [store + "_" + dept for store in stores for dept in depts]
print(state_cats)  # 州名+大类
print("=================================================================")
print(store_depts)  # 商店名（含州）+小类（含大类）

['CA_FOODS', 'CA_HOBBIES', 'CA_HOUSEHOLD', 'TX_FOODS', 'TX_HOBBIES', 'TX_HOUSEHOLD', 'WI_FOODS', 'WI_HOBBIES', 'WI_HOUSEHOLD']
['CA_1_FOODS_1', 'CA_1_FOODS_2', 'CA_1_FOODS_3', 'CA_1_HOBBIES_1', 'CA_1_HOBBIES_2', 'CA_1_HOUSEHOLD_1', 'CA_1_HOUSEHOLD_2', 'CA_2_FOODS_1', 'CA_2_FOODS_2', 'CA_2_FOODS_3', 'CA_2_HOBBIES_1', 'CA_2_HOBBIES_2', 'CA_2_HOUSEHOLD_1', 'CA_2_HOUSEHOLD_2', 'CA_3_FOODS_1', 'CA_3_FOODS_2', 'CA_3_FOODS_3', 'CA_3_HOBBIES_1', 'CA_3_HOBBIES_2', 'CA_3_HOUSEHOLD_1', 'CA_3_HOUSEHOLD_2', 'CA_4_FOODS_1', 'CA_4_FOODS_2', 'CA_4_FOODS_3', 'CA_4_HOBBIES_1', 'CA_4_HOBBIES_2', 'CA_4_HOUSEHOLD_1', 'CA_4_HOUSEHOLD_2', 'TX_1_FOODS_1', 'TX_1_FOODS_2', 'TX_1_FOODS_3', 'TX_1_HOBBIES_1', 'TX_1_HOBBIES_2', 'TX_1_HOUSEHOLD_1', 'TX_1_HOUSEHOLD_2', 'TX_2_FOODS_1', 'TX_2_FOODS_2', 'TX_2_FOODS_3', 'TX_2_HOBBIES_1', 'TX_2_HOBBIES_2', 'TX_2_HOUSEHOLD_1', 'TX_2_HOUSEHOLD_2', 'TX_3_FOODS_1', 'TX_3_FOODS_2', 'TX_3_FOODS_3', 'TX_3_HOBBIES_1', 'TX_3_HOBBIES_2', 'TX_3_HOUSEHOLD_1', 'TX_3_HOUSEHOLD_2', 'WI_

In [8]:
prods = list(train_sales.item_id.unique())  # 商品列表
prod_state = [prod + "_" + state for prod in prods for state in states]  # 商品（含大类和小类）+州名
prod_store = [prod + "_" + store for prod in prods for store in stores]  # 商品（含大类和小类）+商店名（含州）

In [9]:
print("Departments: ", depts)
print("Categories by state: ", state_cats)

Departments:  ['FOODS_1', 'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2']
Categories by state:  ['CA_FOODS', 'CA_HOBBIES', 'CA_HOUSEHOLD', 'TX_FOODS', 'TX_HOBBIES', 'TX_HOUSEHOLD', 'WI_FOODS', 'WI_HOBBIES', 'WI_HOUSEHOLD']


In [10]:
quants = ['0.005', '0.025', '0.165', '0.250', '0.500', '0.750', '0.835', '0.975', '0.995']
days = range(1, 1913 + 29)
time_series_columns = [f'd_{i}' for i in days]

In [11]:
def create_sales(name_list, group):
    '''
    This function returns a dataframe (sales) on the aggregation level given by name list and group
    '''
    # rows_ve = [(name + "_X_" + str(q) + "_" + ve, str(q)) for name in name_list for q in quants for ve in val_eval]
    sales = train_sales.groupby(group)[time_series_columns].sum() #would not be necessary for lowest level
    return sales

In [12]:
def create_quantile_dict(name_list = stores, group = 'store_id' ,X = False):
    '''
    This function writes creates sales data on given aggregation level, and then writes predictions to the global dictionary my_dict
    '''
    sales = create_sales(name_list, group)
    sales = sales.iloc[:, 1857:] #using the last few months data only
    sales_quants = pd.DataFrame(index = sales.index)  # 空ataFrame，初始化为空
    for q in quants:
        sales_quants[q] = np.quantile(sales, float(q), axis = 1)  # 增加分位数的列，如CA_FOODS计算196天的0.005分位数
    full_mean = pd.DataFrame(np.mean(sales, axis = 1))  #  新的DataFrame，只有一列，表示每个index最近196天的均值
    daily_means = pd.DataFrame(index = sales.index)  # 新的DataFrame，初始化为空
    for i in range(7):
        daily_means[str(i)] = np.mean(sales.iloc[:, i::7], axis = 1)   # 新增7列，一周内每天的均值，如最近196天CA_FOOD,周一的均值
        
    daily_factors = daily_means / np.array(full_mean)
    
    daily_factors = pd.concat([daily_factors, daily_factors, daily_factors, daily_factors], axis = 1)
    daily_factors_np = np.array(daily_factors)

    factor_df = pd.DataFrame(daily_factors_np, columns = submission_file.columns[1:])
    factor_df.index = daily_factors.index

    for i,x in enumerate(tqdm(sales_quants.index)):
        for q in quants:
            v = sales_quants.loc[x, q] * np.array(factor_df.loc[x, :])
            if X:
                my_dict[x + "_X_" + q + "_validation"] = v
                my_dict[x + "_X_" + q + "_evaluation"] = v
            else:
                my_dict[x + "_" + q + "_validation"] = v
                my_dict[x + "_" + q + "_evaluation"] = v

In [13]:
my_dict = {}
create_quantile_dict(state_cats, 'state_cat') #6
print(my_dict)


{'CA_FOODS_0.005_validation': array([ 8784.12443358,  8209.91254312,  7878.72892482,  7879.97779331,
        8903.50363482, 11230.51404659, 11625.55362377,  8784.12443358,
        8209.91254312,  7878.72892482,  7879.97779331,  8903.50363482,
       11230.51404659, 11625.55362377,  8784.12443358,  8209.91254312,
        7878.72892482,  7879.97779331,  8903.50363482, 11230.51404659,
       11625.55362377,  8784.12443358,  8209.91254312,  7878.72892482,
        7879.97779331,  8903.50363482, 11230.51404659, 11625.55362377]), 'CA_FOODS_0.005_evaluation': array([ 8784.12443358,  8209.91254312,  7878.72892482,  7879.97779331,
        8903.50363482, 11230.51404659, 11625.55362377,  8784.12443358,
        8209.91254312,  7878.72892482,  7879.97779331,  8903.50363482,
       11230.51404659, 11625.55362377,  8784.12443358,  8209.91254312,
        7878.72892482,  7879.97779331,  8903.50363482, 11230.51404659,
       11625.55362377,  8784.12443358,  8209.91254312,  7878.72892482,
        7879.97

In [14]:
my_dict = {}
#adding prediction to my_dict on all 12 aggregation levels
create_quantile_dict(total, 'Total', X=True) #1
create_quantile_dict(states, 'state_id', X=True) #2
create_quantile_dict(stores, 'store_id', X=True) #3
create_quantile_dict(cats, 'cat_id', X=True) #4
create_quantile_dict(depts, 'dept_id', X=True) #5
create_quantile_dict(state_cats, 'state_cat') #6
create_quantile_dict(state_depts, 'state_dept') #7
create_quantile_dict(store_cats, 'store_cat') #8
create_quantile_dict(store_depts, 'store_dept') #9
create_quantile_dict(prods, 'item_id', X=True) #10
create_quantile_dict(prod_state, 'state_item') #11
create_quantile_dict(prod_store, 'item_store') #12

In [15]:
pred_df = pd.DataFrame(my_dict)
pred_df = pred_df.transpose()
pred_df_reset = pred_df.reset_index()
final_pred = pd.merge(pd.DataFrame(submission_file.id), pred_df_reset, left_on = 'id', right_on = 'index')
del final_pred['index']
final_pred = final_pred.rename(columns={0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7', 7: 'F8', 8: 'F9',
                                        9: 'F10', 10: 'F11', 11: 'F12', 12: 'F13', 13: 'F14', 14: 'F15', 15: 'F16',
                                        16: 'F17', 17: 'F18', 18: 'F19', 19: 'F20', 20: 'F21', 21: 'F22', 
                                        22: 'F23', 23: 'F24', 24: 'F25', 25: 'F26', 26: 'F27', 27: 'F28'})

In [16]:
final_pred = final_pred.fillna(0)

In [17]:
final_pred.to_csv('submission_rf_final.csv', index=False)